In [26]:
import numpy as np
from gensim import matutils
from gensim.models.ldamodel import LdaModel
from sklearn.datasets import fetch_20newsgroups
from gensim.sklearn_api.ldamodel import LdaTransformer
from gensim.models.coherencemodel import CoherenceModel

from sklearn.model_selection import GridSearchCV
import pandas 



In [42]:
def coh_scorer(estimator, X,y=None):
    goodcm = CoherenceModel(model=estimator, texts= data_texts, dictionary=estimator.id2word, coherence='c_v')
    return goodcm.get_coherence()


In [43]:
rand = np.random.mtrand.RandomState(1) # set seed for getting same result
cats = ['rec.sport.baseball', 'sci.crypt']
data = fetch_20newsgroups(subset='train', categories=cats, shuffle=True)

In [45]:
data_texts = [_.split() for _ in data.data]

dfs = []

for keep_n in [250, 500]:
    id2word = Dictionary(data_texts)
    id2word.filter_extremes(no_above=0.8, keep_n=keep_n)
    corpus = [id2word.doc2bow(i.split()) for i in data.data]
    obj = LdaTransformer(id2word=id2word, num_topics=5, iterations=20)
    lda = obj.fit(corpus)
    obj = LdaTransformer(id2word=id2word, 
                         scorer='coh_scorer') # here 'scorer' can be 'perplexity' or 'u_mass'

    parameters = {'num_topics': (2, 3, 5, 10), 'iterations': (1, 20, 50)}

    model = GridSearchCV(obj, parameters, cv=3, scoring=coh_scorer)
    model.fit(corpus)

    df = pd.DataFrame(model.cv_results_)
    df['keep_n'] = keep_n
    dfs.append(df)


ValueError: This topic model is not currently supported. Supported topic models should implement the `get_topics` method.